# Project Overview: Data Normalization for Power BI Modeling
In this notebook, we focus on normalising the cleaned Steam dataset for use in Power BI, following best practices in dimensional modeling (star schema). The aim is to transform the dataset into a structure that enables accurate, scalable, and flexible analysis across developers, genres, publishers, and more.

### Objectives:
* Normalize multi-valued fields (developers, genres, publishers, languages) into clean, analyzable formats

* Create bridge tables to model many-to-many relationships between games and related entities

* Preserve referential integrity between fact and dimension tables

* Prepare clean, one-row-per-entity dimension tables (e.g., developers, genres)

* Align fact tables (prices, reviews, achievements) to the normalized games structure



In [1]:
import pandas as pd
import numpy as np
import sys
import ast
import os
sys.path.append(os.path.abspath(".."))
from utils.data_utils import create_dim_and_bridge, generate_date_dim #utility functions for creating the dimension and bridge tables

In [113]:
prices = pd.read_csv('../data_steam/cleaned/prices_cleaned.csv')
games = pd.read_csv('../data_steam/cleaned/games_cleaned.csv')
achievements = pd.read_csv('../data_steam/cleaned/achievements_cleaned.csv')
reviews = pd.read_csv('../data_steam/cleaned/reviews_cleaned.csv')
history = pd.read_csv('../data_steam/cleaned/history_cleaned.csv')
players = pd.read_csv('../data_steam/cleaned/players_cleaned.csv')

In [114]:
reviews = pd.read_csv('../data_steam/cleaned/reviews_cleaned.csv')
history = pd.read_csv('../data_steam/cleaned/history_cleaned.csv')

### Date
To enable proper time-series analysis in Power BI, we shall create date tables.
The following tables have date columns in them:
* `games` table has a `release_date`: when the game was released, the `release_date` is expected to be one for each game.
* `prices` table has a `date_acquired`: the date when the price was acquired. Each game can have multiple of date_acquired dates since the price can vary over time (for example, during sales)
* `history` table has `date_acquired` : timestamp of when the achievement was earned
* `reviews` table has `posted `: timestamp of when the review was posted
* `players` table has `created`: date of creation of the gaming profile

Above, we have imported a utility function called `generate_date_dim`, which you can find documented in `utils/data_utils.py`.

In short, this function generates a standardised **date dimension table** from any date column in the dataset.  
The output includes the following columns: `date_id`, `Date`, `Year`, `Month`, `MonthNumber`, and `YearMonth`. We do as much in the next cell.


In [3]:
date_release = generate_date_dim(games['release_date'])
date_price = generate_date_dim(prices['date_acquired'])
date_review = generate_date_dim(reviews['posted'])
date_achieved = generate_date_dim(history['date_acquired'])
date_created = generate_date_dim(players['created'])

### Developers and Games normalisation

We will begin with developers and games. As shown below, a single game can be associated with multiple developers  
(this is also true for publishers, genres, and supported languages — we'll handle those later).

Our goal is to create three related tables:
- A `games_dim` table containing unique game information,
- A `developers_dim` table containing unique developer names, and
- A `dev_game_map` bridge table to represent the many-to-many relationship between games and developers.


In [9]:
games[games['gameid']==3255400]

,gameid,title,developers,publishers,genres,supported_languages,release_date,Free to play
17,3255400,Zone Meditation,"['Kevin Wacknov', 'Stuart Heller PhD 7th Dan']",['Mind Body Aware Games LLC'],"['Casual', 'Free To Play', 'Education']",['English'],2024-10-11,Yes


To create the `games_dim` table, we extract a subset of columns from the original `games` table: `gameid`, `title`, and `free_to_play`, resulting in a cleaner and more focused dimension table.

We also generate a `release_date_id` field, which acts as a foreign key linking to the `date_release` dimension table created earlier.  
This surrogate key is included in both tables to support date-based joins in Power BI.


In [6]:
games_dim = games[['gameid', 'title', 'is_free_to_play', 'release_date']].copy()
games_dim.rename(columns={
    'title': 'game_title'
}, inplace=True)
games_dim['release_date'] = pd.to_datetime(games_dim['release_date'], errors='coerce').dropna()
games_dim['release_date_id'] = games_dim['release_date'].dt.strftime('%Y%m%d').astype(int)
games_dim.drop(columns=['release_date'], inplace=True)

In [7]:
games_dim.head()

,gameid,game_title,is_free_to_play,release_date_id
0,3278740,NEURO,No,20241011
1,3270850,Keep Your Eyes Open,No,20241021
2,3267350,Tiny Shooters,Yes,20241019
3,3266470,Futanari Sex Adventures - Episode 5,No,20241017
4,3264110,AUTO_BATTLER_RPG,No,20241022


In the next cell, we use the `create_dim_and_bridge` utility function (documented in `utils/data_utils.py`) to generate both the dimension and bridge tables for the multi-valued fields in the `games` table.

Specifically, we create dimensions and mapping tables for `developers`, `publishers`, `genres`, and `supported_languages`.  
Each call returns a dimension table (e.g., `devs_dim` which has a `developer_id` and `developer_name` - similar case for `genres`, `publishers` and `languages`) and its corresponding bridge table (e.g., `dev_game_map`) to handle the many-to-many relationships.


In [8]:
devs_dim, dev_game_map = create_dim_and_bridge(games, 'developers', 'developer')
publishers_dim, publisher_game_map = create_dim_and_bridge(games, 'publishers', 'publisher')
genres_dim, genre_game_map = create_dim_and_bridge(games, 'genres', 'genre')
langs_dim, lang_game_map = create_dim_and_bridge(games, 'supported_languages', 'language')


In [27]:
devs_dim.head(3) #each developer has a unique ID

,developer_id,developer_name
0,1,Revolt Games
1,2,Texerikus
2,3,madilumar


In [28]:
dev_game_map.head(3)#a game can have more than one developer, and vice versa

,gameid,developer_id
0,3278740,1
1,3270850,2
2,3267350,3


Next, we clean and standardize the date columns in our fact tables — `prices`, `reviews`, `history`, and `players` — to ensure they can properly link to their corresponding date dimension tables.

For each table, we:
- Convert the relevant date column to `datetime` format
- Create a surrogate key called `date_id` in the `YYYYMMDD` format
- Drop the original date column to avoid redundancy, as all date-related attributes will be handled through the associated date dimension

In the example below, we apply this process to the `prices` table to generate `prices_fact`, which links to the `date_price` dimension via `date_id`.


In [9]:
prices_fact = prices.copy()
prices_fact['date_acquired'] = pd.to_datetime(prices_fact['date_acquired'], errors='coerce')
prices_fact['date_id'] = prices_fact['date_acquired'].dt.strftime('%Y%m%d').astype(int)
prices_fact.drop(columns=['date_acquired'], inplace=True)
prices_fact.head(2)

,gameid,usd,date_id
0,3278740,5.99,20241128
1,3270850,3.99,20241128


In [115]:
reviews_fact = reviews.copy()
reviews_fact['posted'] = pd.to_datetime(reviews_fact['posted'], errors='coerce')
reviews_fact['date_id'] = reviews_fact['posted'].dt.strftime('%Y%m%d').astype(int)
reviews_fact.drop(columns=['posted'], inplace=True)
reviews_fact.head(2)

,reviewid,playerid,gameid,helpful,funny,awards,date_id
0,639543,76561198796340888,730,0,0,0,20180322
1,639544,76561198028706627,393380,0,0,0,20250103


In [117]:
history_fact = history.copy()
history_fact['date_acquired'] = pd.to_datetime(history_fact['date_acquired'], errors='coerce')
history_fact['date_id'] = history_fact['date_acquired'].dt.strftime('%Y%m%d').astype(int)
history_fact.drop(columns=['date_acquired'], inplace=True)

In [118]:
players = pd.read_csv('../data_steam/cleaned/players_cleaned.csv')
purchased_games = pd.read_csv('../data_steam/raw/purchased_games.csv')

In [119]:
purchased_games['library'] = purchased_games['library'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
purchased_games.head(2)

,playerid,library
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."


In [103]:
print("Number of rows in players: ",len(players) )
print("Number of unique ids in players: ",players["playerid"].nunique())
print("Number of unique ids in purchased: ",purchased_games["playerid"].nunique())

m = purchased_games[purchased_games['playerid'].isin(players['playerid'])]
len(m)

Number of rows in players:  424683
Number of unique ids in players:  424683
Number of unique ids in purchased:  102548


102548

In [104]:
purchased_games.head(3)

,playerid,library
0,76561198060698936,"[60, 1670, 3830, 1600, 2900, 2910, 2920, 4800,..."
1,76561198287452552,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
2,76561198040436563,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."


In [105]:
purchased_games_exploded = purchased_games.explode('library')
purchased_games_exploded = purchased_games_exploded.rename(columns={'library': 'gameid'})


In [120]:
player_game_map = purchased_games_exploded
player_game_map.head(3)

,playerid,gameid
0,76561198060698936,60
0,76561198060698936,1670
0,76561198060698936,3830


In [142]:
games[games['gameid']==730]

,gameid,title,developers,publishers,genres,supported_languages,release_date,is_free_to_play
58101,730,Counter-Strike 2,['Valve'],['Valve'],"['Action', 'Free To Play']","['Czech', 'Danish', 'Dutch', 'English', 'Finni...",2012-08-21,Yes


In [141]:
#player_game_map['gameid'].nunique()
player_game_map.groupby('gameid')['playerid'].nunique().reset_index(name='num_players').sort_values(by='num_players', ascending=False)

   # .sort_values(by='num_players', ascending=False)


,gameid,num_players
22,730,43967
14750,578080,28356
19,550,24727
2175,218620,23785
4493,304930,21620
...,...,...
40927,3233390,1
40928,3234140,1
40944,3251270,1
40980,3333360,1


In [107]:
player_game_map["playerid"].nunique()

102548

In [ ]:
players['created'] = pd.to_datetime(players['created'], errors='coerce')
players['date_created_id'] = players['created'].dt.strftime('%Y%m%d').astype('Int64')
players_dim = players[['playerid', 'country', 'date_created_id']].drop_duplicates().reset_index(drop=True)
player_game_map['playerid'] = player_game_map['playerid'].astype(int)

In [121]:

player_id_map = players_dim[['playerid']].drop_duplicates().reset_index(drop=True)
player_id_map['custom_player_id'] = range(1, len(player_id_map) + 1)
players_dim = players_dim.merge(player_id_map, on='playerid', how='left')
players_dim.head(3)

,playerid,country,created_date_id,custom_player_id
0,76561198287452552,Brazil,20160302,1
1,76561198040436563,Israel,20110410,2
2,76561198049686270,Unknown,20110928,3


In [122]:
reviews_fact = reviews_fact.merge(player_id_map, on='playerid', how='left')
reviews_fact.head(2)

,reviewid,playerid,gameid,helpful,funny,awards,date_id,custom_player_id
0,639543,76561198796340888,730,0,0,0,20180322,255949
1,639544,76561198028706627,393380,0,0,0,20250103,255968


In [124]:
history_fact = history_fact.merge(player_id_map, on='playerid', how='left')
player_game_map = player_game_map.merge(player_id_map, on='playerid', how='left')


In [135]:
# Step 1: Unique counts
players_dim_unique = players_dim['playerid'].nunique()
player_game_map_unique = player_game_map['playerid'].nunique()

# Step 2: How many playerids in player_game_map also exist in players_dim?
shared_unique_playerids = player_game_map[player_game_map['playerid'].isin(players_dim['playerid'])]['playerid'].nunique()


# Step 3: Show differences and summary
print("Unique playerid count in players_dim:", players_dim_unique)
print("Unique playerid count in player_game_map:", player_game_map_unique)
print("Shared unique playerids:", shared_unique_playerids)
print("Missing playerids:", player_game_map_unique - shared_unique_playerids)


Unique playerid count in players_dim: 424683
Unique playerid count in player_game_map: 102548
Shared unique playerids: 102548
Missing playerids: 0


In [127]:
# Drop original playerid and rename custom one back to 'playerid'
for df in [players_dim, reviews_fact, history_fact, player_game_map]:
    df.drop(columns=['playerid'], inplace=True)
    df.rename(columns={'custom_player_id': 'playerid'}, inplace=True)


## Summary
We have now transformed a messy raw dataset with multi-valued fields and mixed date logic into a clean, Power BI–ready star schema. The importance of getting the data into this form cannot be understated, especially if we want to use Power BI's full capabilities seamlessly.
These are our tables now:
#### Fact Tables:

* prices_fact: Game price observations over time

* reviews_fact: Player reviews with helpful/funny/awards

* history_fact: When players unlocked achievements

#### Dimension Tables:

* games_dim: Game metadata with release date, F2P flag

* players_dim: Player IDs, country, account creation

* achievements_dim: Achievement details per game

* devs_dim, genres_dim, publishers_dim, languages_dim: Normalised lists

#### Bridge Tables (many-to-many):

* `dev_game_map`, `genre_game_map`, `publisher_game_map`, `language_game_map`

#### Date Dimensions:

* `date_release`, `date_price`, `date_review`, `date_achieved`, `date_created`
Each has date_id, Year, Month, etc.

## Utility functions
Apart from the Power BI setting up,  reusable tilities were created and used
* `create_dim_and_bridge()` — turns any multi-valued column into a dimension + bridge table

* `generate_date_dim()` — creates reusable date dimension from any datetime series

Stored in `utils/data_utils.py`, imported into notebooks



In [36]:
date_review.head()

,date_id,Date,Year,Month,MonthNumber,YearMonth
0,20101015,2010-10-15,2010,Oct,10,2010-10
1,20101016,2010-10-16,2010,Oct,10,2010-10
2,20101017,2010-10-17,2010,Oct,10,2010-10
3,20101018,2010-10-18,2010,Oct,10,2010-10
4,20101019,2010-10-19,2010,Oct,10,2010-10


In [145]:
player_game_map.groupby('gameid')['playerid'].nunique().reset_index(name='num_players').sort_values(by='num_players', ascending=False)[:2]

,gameid,num_players
22,730,43967
14750,578080,28356
19,550,24727


In [148]:
games[games['title']=='Escape']

,gameid,title,developers,publishers,genres,supported_languages,release_date,is_free_to_play
15030,1835250,Escape,['criswei'],['criswei'],"['Adventure', 'Indie']",['English'],2022-02-07,No
17756,2092070,Escape,['Windshield Wiper Max Games'],['Windshield Wiper Max Games'],"['Casual', 'Free To Play']",['English'],2022-08-16,Yes
29548,759010,Escape,['Ragdoll Inc'],['Ragdoll Inc'],"['Action', 'Adventure', 'Indie']",['English'],2018-01-28,Yes
77115,365050,Escape,['Stephane Bottin'],['Stephane Bottin'],"['Free To Play', 'Indie', 'Strategy']",['English'],2015-04-20,Yes
91125,1607470,Escape,['CheYne_CY'],['CheYne_CY'],"['Adventure', 'Indie']","['English', 'Simplified Chinese']",2021-05-05,No


In [149]:
reviews_fact.head(2)


,reviewid,gameid,helpful,funny,awards,review_date_id,playerid
0,639543,730,0,0,0,20180322,255949
1,639544,393380,0,0,0,20250103,255968


In [154]:
#reviews_fact.head()
reviews_fact.groupby('gameid')['reviewid'].nunique().reset_index(name='num_reviews').sort_values(by='num_reviews', ascending=False)[:9]

,gameid,num_reviews
26,730,70529
23,570,15759
10923,578080,11743
20,440,11409
2473,271590,10812
4957,359550,8871
2016,252490,8370
128,4000,7813
2032,252950,7142


In [130]:
#players_dim.rename(columns={'date_created_id': 'created_date_id'}, inplace=True)
players_dim.to_csv("../data_steam/normalised/players_dim.csv", index=False)
player_game_map.to_csv("../data_steam/normalised/player_game_map.csv", index=False) 
reviews_fact.rename(columns={'date_id': 'review_date_id'}, inplace=True)
history_fact.rename(columns={'date_id': 'achieved_date_id'}, inplace=True)

In [83]:
players_dim[players_dim["playerid"]=="76561200000000000"]
#players_dim["playerid"][0]

,playerid,country,created_date_id


In [13]:
prices_fact.rename(columns={'date_id': 'price_date_id'}, inplace=True)
reviews_fact.rename(columns={'date_id': 'review_date_id'}, inplace=True)
history_fact.rename(columns={'date_id': 'achieved_date_id'}, inplace=True)
players_dim.rename(columns={'date_created_id': 'created_date_id'}, inplace=True)

In [14]:
# Export dimensions
games_dim.to_csv("../data_steam/normalised/games_dim.csv", index=False)
devs_dim.to_csv("../data_steam/normalised/devs_dim.csv", index=False)
publishers_dim.to_csv("../data_steam/normalised/publishers_dim.csv", index=False)
genres_dim.to_csv("../data_steam/normalised/genres_dim.csv", index=False)
langs_dim.to_csv("../data_steam/normalised/languages_dim.csv", index=False)
players_dim.to_csv("../data_steam/normalised/players_dim.csv", index=False)
achievements.to_csv("../data_steam/normalised/achievements_dim.csv", index=False)

# Export bridge tables
dev_game_map.to_csv("../data_steam/normalised/dev_game_map.csv", index=False)
publisher_game_map.to_csv("../data_steam/normalised/publisher_game_map.csv", index=False)
genre_game_map.to_csv("../data_steam/normalised/genre_game_map.csv", index=False)
lang_game_map.to_csv("../data_steam/normalised/language_game_map.csv", index=False)
player_game_map.to_csv("../data_steam/normalised/player_game_map.csv", index=False)  
# Export fact tables
prices_fact.to_csv("../data_steam/normalised/prices_fact.csv", index=False)
reviews_fact.to_csv("../data_steam/normalised/reviews_fact.csv", index=False)
history_fact.to_csv("../data_steam/normalised/history_fact.csv", index=False)

# Export date tables
date_release.to_csv("../data_steam/normalised/date_release.csv", index=False)
date_price.to_csv("../data_steam/normalised/date_price.csv", index=False)
date_review.to_csv("../data_steam/normalised/date_review.csv", index=False)
date_achieved.to_csv("../data_steam/normalised/date_achieved.csv", index=False)
date_created.to_csv("../data_steam/normalised/date_created.csv", index=False)  


In [37]:
games_dim.head()

,gameid,game_title,is_free_to_play,release_date_id
0,3278740,NEURO,No,20241011
1,3270850,Keep Your Eyes Open,No,20241021
2,3267350,Tiny Shooters,Yes,20241019
3,3266470,Futanari Sex Adventures - Episode 5,No,20241017
4,3264110,AUTO_BATTLER_RPG,No,20241022


### 🔍 Debugging Publisher Filter Behavior in Power BI

During dashboard development, we noticed that selecting a publisher in the "Top Publishers" bar chart correctly filtered the F2P summary table, but had no effect on key KPI cards or detailed tables (e.g., games and prices).

This discrepancy highlighted a key insight:

> While visuals using fields from `publishers_dim` or direct aggregations from `publisher_game_map` responded correctly, any visual relying on `games_dim` or `prices_fact` remained unaffected.

#### ✅ Root Cause:
The relationship between `publisher_game_map` and `games_dim` was set to **single-directional filtering**, flowing only from `games_dim → publisher_game_map`. This prevented filters from `publishers_dim` from propagating down to `games_dim` and, by extension, to `prices_fact`.

#### 🛠 Solution:
We updated the relationship between `publisher_game_map` and `games_dim` to use **bidirectional filtering**. This allowed the publisher filter to flow through the bridge table and into the relevant fact tables.

#### 📌 Lesson:
When using bridge tables to model many-to-many relationships (e.g., games with multiple publishers), ensure that **cross filter direction is set to "Both"** if downstream filtering across multiple related tables is expected in visuals.

This fix restored full interactivity across KPIs, tables, and charts using the publisher selection.
